# Deploy model to prod environment

## dev account

Login AWS console:

* aws login account: mlops-dev
* user: mlops-dev-developers

Open the cloud9 - my-mlops-works

## Deploy model to prod script

* mkdir deploy-model-to-prod
* aws --profile mlops-devops-developers codecommit create-repository --repository-name deploy-model-to-prod
* cd deploy-model-to-prod
* git init
* git remote add origin https://git-codecommit.us-west-2.amazonaws.com/v1/repos/deploy-model-to-prod
* git remote -v
* touch deploy-model-to-prod.py

Paste the following codes and commit the code into codecommit

## deploy-model-to-prod.py

In [ ]:
import boto3, sys

prod_access_key = sys.argv[1]
prod_secrete_key = sys.argv[2]
model_name = sys.argv[3]

# devops sagemaker
sm = boto3.client('sagemaker')
resp = sm.describe_model(ModelName=model_name)

# prod sagemaker
sess_prod = boto3.Session(aws_access_key_id=prod_access_key, aws_secret_access_key=prod_secrete_key)
sm_prod = sess_prod.client('sagemaker')
prod_account = sess_prod.client('sts').get_caller_identity()['Account']

image = resp['PrimaryContainer']['Image']
modelDataUrl = resp['PrimaryContainer']['ModelDataUrl']
model_arn = resp['ModelArn']
role = 'arn:aws:iam::{}:role/SageMakerExecutionRole'.format(prod_account)

# create prod model
create_model_response = sm_prod.create_model(
    ModelName=model_name,
    ExecutionRoleArn=role,
    PrimaryContainer={
        'Image': image,
        'ModelDataUrl': modelDataUrl},
    Tags=[{"Key": "model_arn","Value": model_arn}])

print(create_model_response['ModelArn'])

## Config prod account

Login AWS console:

aws login account: mlops-prod
user: mlops-prod-admin

##